In [1]:
import tensorflow as tf
import gym
import numpy as np

In [2]:
env = gym.make("FrozenLake-v0")
num_obs = env.observation_space.n
num_acts = env.action_space.n

[2017-05-14 13:34:13,711] Making new env: FrozenLake-v0


In [3]:
sratio=[]

In [21]:
tf.reset_default_graph() # no clue what this does

## cries
I dont get it. This thing gave me a decent .45-.55 ratio as a single layer NN.<br>
Obvious step to try add a hidden layer, what does it do? die and give me .0 ratio

In [33]:
# These lines establish the feed-forward part of the network used to choose actions

inputs = tf.placeholder(shape=[1,num_obs], dtype=tf.float32) # the input value X
nextQ = tf.placeholder(shape=[1,num_acts],dtype=tf.float32) # the target value Y

# the weights - 2 layers
W_ = tf.Variable(tf.random_uniform([num_obs,10],0,0.01)) # 16x10
L1 = tf.nn.relu(tf.matmul(inputs,W_))
W = tf.Variable(tf.random_uniform([10,num_acts],0,0.01)) #10x4

Qout = tf.matmul(L1,W) #hypothesis?
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference
# between the target and prediction Q values

# the above value is computed by using the q-learning bellman equations
loss = tf.reduce_sum(tf.square(nextQ-Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [34]:
init = tf.global_variables_initializer()

# Set learning parameters
y = .99
e = 0.5

In [35]:
def run_episode(s):
        global e
        j = 0
        rAll =0
        #The Q-Network
        while j < 99: #to check for timeout
            j+=1
            #Choose an action with e chance of random action from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs:np.identity(num_obs)[s:s+1]})
            #print(np.identity(num_obs)[s:s+1]) # I think a one-hot vector representation, as obs are discrete
            if np.random.rand(1) < e:
                a[0] = env.action_space.sample() # the randomly chosen action

            #Get new state and reward from environment
            s1,r,d,_ = env.step(a[0])
            
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs:np.identity(num_obs)[s1:s1+1]})#returns 1x4 of q' values
            
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = r + y*maxQ1 #a[0] is the index of the chosen action
            
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W_],feed_dict={inputs:np.identity(num_obs)[s:s+1],nextQ:targetQ})
            rAll += r
            s = s1
            if d == True:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        return rAll, j

In [36]:
num_episodes = 5000
#create lists to contain total rewards and steps per episode
jList = []
rList = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation
        s = env.reset()
        result,iter_num = run_episode(s)
        jList.append(iter_num)
        rList.append(result)

In [37]:
print ("Percent of succesful episodes: " + str(sum(rList)/num_episodes) + "%")
sratio.append(sum(rList)/num_episodes)

Percent of succesful episodes: 0.0002%


In [32]:
print(sratio)

[0.0012, 0.022, 0.0, 0.4794]
